https://codes.ecmwf.int/grib/param-db/ \
https://cds.climate.copernicus.eu/datasets/reanalysis-era5-pressure-levels?tab=download

In [1]:
import cdsapi
import numpy as np
import netCDF4
import os

# Function to read area specifications from namelist.wps
def read_area_from_namelist(namelist_path):
    area = {}
    with open(namelist_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if 'ref_lat' in line:
                area['ref_lat'] = float(line.split('=')[1].strip())
            elif 'ref_lon' in line:
                area['ref_lon'] = float(line.split('=')[1].strip())
            elif 'e_we' in line:
                area['e_we'] = int(line.split('=')[1].strip())
            elif 'e_sn' in line:
                area['e_sn'] = int(line.split('=')[1].strip())
    return area

# Function to download ERA5 data
def download_era5_data(start_date, end_date, time_step, pressure_levels, variables, area, output_dir):
    c = cdsapi.Client()
    dataset = 'reanalysis-era5-pressure-levels' if pressure_levels else 'reanalysis-era5-single-levels'
    request = {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': variables,
        'year': [str(year) for year in range(start_date[0], end_date[0] + 1)],
        'month': [f'{month:02d}' for month in range(start_date[1], end_date[1] + 1)],
        'day': [f'{day:02d}' for day in range(start_date[2], end_date[2] + 1)],
        'time': [f'{hour:02d}:00' for hour in range(0, 24, time_step)],
        'pressure_level': pressure_levels if pressure_levels else None,
        'area': [area['ref_lat'] + area['e_sn'] * 0.25, area['ref_lon'] - area['e_we'] * 0.25,
                 area['ref_lat'] - area['e_sn'] * 0.25, area['ref_lon'] + area['e_we'] * 0.25],
        'grid': [0.25, 0.25]
    }
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        target = os.path.join(output_dir, f'{start_date}{end_date}_all.nc')
        c.retrieve(dataset, request, target)
    else:
        target = os.path.join(output_dir, f'{start_date}{end_date}_all.nc')
        c.retrieve(dataset, request, target)

# Download function
def download_main(variables, pressure_levels,output_dir):
    # Read area from namelist.wps
    area = read_area_from_namelist('namelist_larger.wps')
    print([area['ref_lat'] + area['e_sn'] * 0.25, area['ref_lon'] - area['e_we'] * 0.25, \
        area['ref_lat'] - area['e_sn'] * 0.25, area['ref_lon'] + area['e_we'] * 0.25])
    # Define download parameters
    start_date = (2024, 9, 4)  # Year, Month, Day
    end_date = (2024, 9, 7)
    time_step = 1  # Hours
    # Download data
    for date in range(start_date[2],end_date[2]+1):
        day = (2024,9,date)
        # download_era5_data(day, day, time_step, pressure_levels, variables, area, output_dir)

pressure_levels = ['1000', '925', '850', '700', '600', '500', '400', '300', '200', '100']
variables = [
        "geopotential",
        "specific_humidity",
        "temperature",
        "u_component_of_wind",
        "v_component_of_wind",
        "vertical_velocity",
        "vorticity"
    ]
output_dir = 'era5_data'

download_main(variables,pressure_levels,output_dir)

[31.449, 81.805, 6.949000000000002, 145.805]


In [8]:

variable_mapping = {
    'u_component_of_wind': 'u',
    'v_component_of_wind': 'v',
    'temperature': 't',
    'geopotential': 'z',
    'dew_point_temperature': 'd',
    'specific_humidity': 'q',
    'relative_humidity': 'r',
    'vorticity':'vo',
    'vertical_velocity':'w'
}
start_date = (2024, 9, 3)  # Year, Month, Day
end_date = (2024, 9, 7)
# Function to convert NetCDF to NumPy .dat
def convert_netcdf_to_numpy(variables, output_dir):
    # Convert each NetCDF file to NumPy .dat
    
    for date in range(start_date[2],end_date[2]+1):
        day = (2024,9,date)
        netcdf_file = os.path.join(output_dir, f'{day}{day}_all.nc')
        with netCDF4.Dataset(netcdf_file, 'r') as nc:
            for var in variables:
                output_file = os.path.join(output_dir, f'{day}{var}.dat')
                data = np.array(nc.variables[variable_mapping[var]][:]).astype(np.float64)
                data = np.transpose(data, axes=(3, 2, 1, 0))
                data.flatten(order='F').tofile(output_file)
    print(data.shape)

        

convert_netcdf_to_numpy(variables,output_dir)

(257, 99, 10, 24)


In [ ]:
import os
import numpy as np

# Function to concatenate .dat files for each variable
def concatenate_dat_files():
    for var in variables:
        # Create a list to hold data for each date
        all_data = []
        
        # Process each date
        for date in range(start_date[2], end_date[2] + 1):
            day = (2024, 9, date)
            input_file = os.path.join(output_dir, f'{day}{var}.dat')
            
            try:
                # Read data from file
                data = np.fromfile(input_file, dtype=np.float64)
                all_data.append(data)
                print(f"Added {input_file} to concatenation list")
            except FileNotFoundError:
                print(f"Warning: File {input_file} not found, skipping")
        
        if all_data:
            # Concatenate all data for this variable
            combined_data = np.concatenate(all_data)
            
            # Write to output file
            output_file = os.path.join(output_dir, f'combined_{var}.dat')
            combined_data.tofile(output_file)
            print(all_data.shape)
            print(f"Created combined file: {output_file}")
        else:
            print(f"No data found for variable {var}")

# Execute the concatenation
concatenate_dat_files()

Added era5_data/(2024, 9, 3)geopotential.dat to concatenation list
Added era5_data/(2024, 9, 4)geopotential.dat to concatenation list
Added era5_data/(2024, 9, 5)geopotential.dat to concatenation list
Added era5_data/(2024, 9, 6)geopotential.dat to concatenation list
Added era5_data/(2024, 9, 7)geopotential.dat to concatenation list
Created combined file: era5_data/combined_geopotential.dat
Added era5_data/(2024, 9, 3)specific_humidity.dat to concatenation list
Added era5_data/(2024, 9, 4)specific_humidity.dat to concatenation list
Added era5_data/(2024, 9, 5)specific_humidity.dat to concatenation list
Added era5_data/(2024, 9, 6)specific_humidity.dat to concatenation list
Added era5_data/(2024, 9, 7)specific_humidity.dat to concatenation list
Created combined file: era5_data/combined_specific_humidity.dat
Added era5_data/(2024, 9, 3)temperature.dat to concatenation list
Added era5_data/(2024, 9, 4)temperature.dat to concatenation list
Added era5_data/(2024, 9, 5)temperature.dat to con